In [0]:

%matplotlib inline

import os
import pickle
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Input, Activation, Add, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Convolution3D, MaxPooling3D, GlobalAveragePooling3D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

from tensorflow.examples.tutorials.mnist import input_data

In [7]:
from google.colab import drive
drive.mount('./gdrive')

Drive already mounted at ./gdrive; to attempt to forcibly remount, call drive.mount("./gdrive", force_remount=True).


In [38]:
%cd "gdrive/My Drive/Colab Notebooks"

/content/gdrive/My Drive/Colab Notebooks


In [0]:
def Shortcut3D(input_data, fx, data_format = "channels_first"):

  # チャンネル数の取得
  if data_format == "channels_first":
    channel_num = int(fx.shape[1])
  else:
    channel_num = int(fx.shape[4])
      
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  buf = Convolution3D(channel_num, (1,1,1), strides=(1,1,1), padding='valid', data_format=data_format)(input_data)

  # 2つを足す
  return Add()([buf, fx])

In [0]:
def Resblock3D(n_filters, strides=(1,1,1), data_format = "channels_first", axis=1):
  def f(input_data):   
    fx = Convolution3D(n_filters, (3,3,3), strides=strides, kernel_initializer='he_normal', padding='same', data_format=data_format)(input_data)
    fx = BatchNormalization(axis=axis)(fx)
    fx = Activation('relu')(fx)
    fx = Convolution3D(n_filters, (3,3,3), strides=strides, kernel_initializer='he_normal', padding='same', data_format=data_format)(fx)
    fx = BatchNormalization(axis=axis)(fx)

    return Shortcut3D(input_data, fx, data_format)

  return f

In [0]:
def Resnet3D(channel = 3, frame = 10, height = 64, width = 64, num_classes = 2, is_channels_first = True):
  if is_channels_first:
    data_format = "channels_first"
    axis=1
    input_data = Input(shape=(channel, frame, height, width))
  else:
    data_format = "channels_last"
    axis=-1
    input_data = Input(shape=(frame, height, width, channel))
  
  print(input_data.shape)

  x = Convolution3D(32, (7,7,7), strides=(1,1,1), kernel_initializer='he_normal', padding='same', data_format=data_format)(input_data)
  x = BatchNormalization(axis=axis)(x)
  x = Activation('relu')(x)
  x = MaxPooling3D((3, 3,3), strides=(2,2,2), padding='same', data_format=data_format)(x)


  x = Resblock3D(n_filters=64, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=64, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=64, data_format = data_format, axis=axis)(x)
  x = MaxPooling3D(strides=(2,2,2), data_format=data_format)(x)  
  x = Resblock3D(n_filters=128, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=128, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=128, data_format = data_format, axis=axis)(x)


  x =  GlobalAveragePooling3D(data_format=data_format)(x)
  x = Dense(num_classes, kernel_initializer='he_normal', activation='softmax')(x)

  model = Model(inputs=input_data, outputs=x)
  return model

In [0]:
is_channels_first = False

if is_channels_first:
    with open('./resnet3d_data_x_channel_first.pkl', 'rb') as fx:
        X = pickle.load(fx)
    
    with open('./resnet3d_data_y_channel_first.pkl', 'rb') as fy:
        Y = pickle.load(fy)
else:
    with open('./resnet3d_data_x_channel_last.pkl', 'rb') as fx:
        X = pickle.load(fx)
    
    with open('./resnet3d_data_y_channel_last.pkl', 'rb') as fy:
        Y = pickle.load(fy)    
        
Y = keras.utils.to_categorical(Y, 2)
X_train, X_val,  Y_train,  Y_val = train_test_split(X, Y, train_size=0.8, shuffle=True)
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, train_size=0.8, shuffle=True)

In [48]:
model = Resnet3D(channel=3, frame=10, height=64, width=64, num_classes=2, is_channels_first=is_channels_first)
adam = keras.optimizers.Adam()
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

(?, 10, 64, 64, 3)


In [49]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 10, 64, 64,  0                                            
__________________________________________________________________________________________________
conv3d_76 (Conv3D)              (None, 10, 64, 64, 3 32960       input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_52 (BatchNo (None, 10, 64, 64, 3 128         conv3d_76[0][0]                  
__________________________________________________________________________________________________
activation_28 (Activation)      (None, 10, 64, 64, 3 0           batch_normalization_52[0][0]     
____________________________________________________________________________________________

In [46]:
history = model.fit(X_train, Y_train,
                    batch_size=256,
                    epochs=50,
                    verbose=1,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1)],
                    validation_data=(X_test, Y_test))

Train on 1034 samples, validate on 259 samples
Epoch 1/50
1034/1034 [==============================] - 57s 55ms/sample - loss: 2.5968 - acc: 0.5667 - val_loss: 22.7038 - val_acc: 0.6216
Epoch 2/50
1034/1034 [==============================] - 39s 38ms/sample - loss: 0.7969 - acc: 0.5648 - val_loss: 27.8626 - val_acc: 0.3784
Epoch 3/50
1034/1034 [==============================] - 40s 38ms/sample - loss: 0.7837 - acc: 0.5687 - val_loss: 147.4382 - val_acc: 0.3784
Epoch 4/50
1034/1034 [==============================] - 39s 38ms/sample - loss: 0.7009 - acc: 0.5039 - val_loss: 120.1122 - val_acc: 0.6216
Epoch 5/50
1034/1034 [==============================] - 40s 38ms/sample - loss: 0.7252 - acc: 0.6441 - val_loss: 401.8571 - val_acc: 0.6216
Epoch 6/50
1034/1034 [==============================] - 40s 38ms/sample - loss: 0.6874 - acc: 0.4855 - val_loss: 137.4836 - val_acc: 0.6216
Epoch 7/50
1034/1034 [==============================] - 40s 38ms/sample - loss: 0.6584 - acc: 0.6441 - val_loss: 68